In [1]:
import numpy as np

from src.dataset import *
from src.model4 import *
from src.train import *

import torch
import src.templates
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "template_TNR"

C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = QM9Dataset(root="./dataset/qm9",
                     target_y="gap",
                     max_atoms=100,
                     force_reload=False
                     )

In [11]:
model3 = E3GG(node_attr_dim = dataset.node_attr.shape[1],
              edge_dim   = dataset.edge_attr.shape[1],
              hidden_dim = 64,
              num_layers=7,
              equivariant=False
              )

In [4]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import torch

# split
n = len(dataset)
n_train = int(0.90 * n)
n_test = n - n_train
train_ds, test_ds = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

In [5]:
n_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = torch.optim.Adam(model3.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

train_losses, val_losses = [], []

In [6]:
model3.to(device)

_train_losses, _val_losses = train( model = model3,
                                  train_loader = train_loader,
                                  val_loader = test_loader,
                                  optimizer = optimizer,
                                  scheduler = lr_scheduler,
                                  epochs = n_epochs,
                                  device = device,
                                  )
train_losses += _train_losses
val_losses += _val_losses

100%|██████████| 500/500 [3:44:30<00:00, 26.94s/it]  


In [29]:
fig = go.Figure(layout={
        'plot_bgcolor': 'white',
        'paper_bgcolor' : 'white',})

fig.update_layout(width = 600,
                  height = 600,
                  legend = dict(x = 0.95, y = 0.9)
                  )

fig.update_xaxes(title = "epoch")
fig.update_yaxes(title = "MSE loss")

fig.add_trace(go.Scatter(y=val_losses, mode='lines', name = 'Validation loss'))
fig.add_trace(go.Scatter(y=train_losses, mode='lines', name = 'Training loss'))

In [8]:
import src.templates
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "template_TNR"

model3.to("cpu")

fig = go.Figure(layout={
        'plot_bgcolor': 'white',
        'paper_bgcolor' : 'white',})

fig.update_layout(width = 600,
                  height = 600
                  )
fig.update_xaxes(title = "model",
                 range=[0, 15]
)
fig.update_yaxes(title = "dataset",
                 range=[0, 15])

fig.add_trace(go.Scatter(x = [-15, 15], y = [-15, 15], mode = 'lines', line=dict(color = "lightgrey"), showlegend=False))

for batch in tqdm(train_loader):
    fig.add_trace(go.Scatter(x = model3(batch).detach(),
                             y = batch.y,
                             mode = 'markers',
                             marker = dict(color = "blue"),
                             showlegend=False,
                             )
                  )

for batch in tqdm(test_loader):
    fig.add_trace(go.Scatter(x = model3(batch).detach(),
                             y = batch.y,
                             mode = 'markers',
                             marker = dict(color = "green"),
                             showlegend=False,
                             )
                  )

fig.show()

100%|██████████| 103/103 [00:02<00:00, 37.57it/s]


In [9]:
torch.save(model3.state_dict(), "model.pt")

In [ ]:
model3 = E3GG(node_attr_dim = dataset.node_attr.shape[1],
              edge_dim   = dataset.edge_attr.shape[1],
             hidden_dim = 32)
model3.load_state_dict(torch.load("model.pt", map_location="cpu"))
model3.eval()

In [26]:
MAE_train = []
MAE_valid = []

for batch in train_loader:
    MAE_train.append( (np.abs(model3(batch).detach() -  batch.y)).numpy() )

for batch in test_loader:
    MAE_valid.append( (np.abs(model3(batch).detach() -  batch.y)).numpy() )

C:\Users\alwas\AppData\Local\Temp\ipykernel_8632\681261833.py:5: DeprecationWarning:

__array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)

C:\Users\alwas\AppData\Local\Temp\ipykernel_8632\681261833.py:8: DeprecationWarning:

__array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)



MAE (train): 0.08328903466463089
MAE (valid): 0.10394870489835739


In [28]:
print(f"MAE (train): {1000 * np.mean(np.concatenate(MAE_train)):.2f} meV")
print(f"MAE (valid): {1000 * np.mean(np.concatenate(MAE_valid)):.2f} meV")

MAE (train): 83.29 meV
MAE (valid): 103.95 meV


In [7]:
np.sum([p.numel() for p in model3.parameters()])


np.int64(219087)